# Statistical Analysis of TDA derived signals

following https://towardsdatascience.com/a-quick-introduction-on-granger-causality-testing-for-time-series-analysis-7113dc9420d2

In [8]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date


from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import config as c 

key, secret = c.apis[1][0], c.apis[1][1]
client = Client(key, secret)

In [11]:
norm = np.loadtxt("orderflow_norms.txt", dtype = float)
def parse(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f")
with open("y_sw.txt", "r") as w:
    y_sw = np.array([np.datetime64(parse(i.strip()[:-3])) for i in w.readlines()])


columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
            "Close Time", "Quote Asset Volume", "Number of Trades",
            "Taker Buy Base Volume", "Taker Buy Quote Asset Volume",
            "Ignore"]
period = "1h"



# df of norm, and any other stats to be plotted
calced_df = pd.DataFrame(
    {
        "norm": norm[:,1][1:],
    },
    index = y_sw[1:]
)

# range of data to work with
start = calced_df.index[0].replace(microsecond=0, second=0, minute=0)
end = calced_df.index[-1].replace(microsecond=0, second=0, minute=0, hour = calced_df.index[-1].hour)
idx = pd.date_range(start=start, end=end, freq = period)


# reindex calced_df to be evenly spaces
calced_df = calced_df.to_period(freq = period).groupby(calced_df.to_period(freq = period).index).mean().to_timestamp().reindex(idx, fill_value=0)



# data to download from binance
klines = client.get_historical_klines("BTCUSDT", period, str(start.timestamp() * 1000-3600000), str(end.timestamp() * 1000))
klines = [[float(i) for i in line] for line in klines ]
k_df = pd.DataFrame(klines, columns = columns, index = idx)

In [12]:
close_series = k_df["Close"]
norm_series = pd.Series(calced_df["norm"].values/max(calced_df["norm"].values), name = 'norm', index = idx)


In [13]:
n_obs = 20
# df_train, df_test = df[0:-n_obs], df[-n_obs:]

from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
        
print('ADF Test: Close time series')
adf_test(close_series)

print('ADF Test: Norm time series')
adf_test(norm_series)

ADF Test: Close time series
ADF Statistics: -1.765038
p-value: 0.397952
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568
ADF Test: Norm time series
ADF Statistics: -9.992389
p-value: 0.000000
Critical values:
	1%: -3.436
	5%: -2.864
	10%: -2.568


In [14]:
# need to make df with differenced close values, ignore first value

df = pd.concat([np.log(close_series.pct_change()+1), norm_series], axis=1)[1:]

In [15]:
df

,Close,norm
2020-01-04 00:00:00,-0.001674,0.039050
2020-01-04 01:00:00,0.000016,0.038422
2020-01-04 02:00:00,-0.003559,0.036837
2020-01-04 03:00:00,-0.004656,0.038174
2020-01-04 04:00:00,0.002408,0.038174
...,...,...
2020-02-17 12:00:00,-0.002135,0.119927
2020-02-17 13:00:00,0.005045,0.040601
2020-02-17 14:00:00,-0.001027,0.075107
2020-02-17 15:00:00,-0.007914,0.226376


In [16]:
df_train, df_test = df[0:-n_obs], df[-n_obs:]
df_train_transformed = df_train

print('ADF Test: Close time series')
adf_test(df_train_transformed['Close'])

print('ADF Test: Norm time series')
adf_test(df_train_transformed['norm'])

# reject null hypo, differenced time sreies is now stationary. now make df with differenced s&p and norm data

ADF Test: Close time series
ADF Statistics: -32.815050
p-value: 0.000000
Critical values:
	1%: -3.437
	5%: -2.864
	10%: -2.568
ADF Test: Norm time series
ADF Statistics: -10.323076
p-value: 0.000000
Critical values:
	1%: -3.437
	5%: -2.864
	10%: -2.568


In [17]:
from statsmodels.tsa.api import VAR

model = VAR(df_train_transformed)
for i in range(0, 10):
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 0
AIC :  -15.98343354525683
BIC :  -15.974013984929888
FPE :  1.1441501185340181e-07
HQIC:  -15.979862441553347 

Lag Order = 1
AIC :  -16.68981396931544
BIC :  -16.661533845399294
FPE :  5.6455453782583786e-08
HQIC:  -16.67909203185351 

Lag Order = 2
AIC :  -16.713061910344496
BIC :  -16.665892572848048
FPE :  5.5158125542870344e-08
HQIC:  -16.69517761278913 

Lag Order = 3
AIC :  -16.707670252256047
BIC :  -16.641582979680695
FPE :  5.5456336409839716e-08
HQIC:  -16.682612038742665 

Lag Order = 4
AIC :  -16.704366399736465
BIC :  -16.619332398829762
FPE :  5.563988383746747e-08
HQIC:  -16.672122684767647 

Lag Order = 5
AIC :  -16.701104493624914
BIC :  -16.59709489913314
FPE :  5.5821711307376344e-08
HQIC:  -16.66166366196671 

Lag Order = 6
AIC :  -16.70958900176638
BIC :  -16.586574876186024
FPE :  5.5350151025118273e-08
HQIC:  -16.662939408344133 

Lag Order = 7
AIC :  -16.70557583211378
BIC :  -16.563528165441937
FPE :  5.557280358065448e-08
HQIC:  -16.651705801907

In [18]:
# model 2 has least aic and bic, so thats probably the best model

model.select_order(2)
results = model.fit(maxlags=2, ic='aic')
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 29, Aug, 2021
Time:                     11:21:54
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -16.6659
Nobs:                     1051.00    HQIC:                  -16.6952
Log likelihood:           5810.11    FPE:                5.51581e-08
AIC:                     -16.7131    Det(Omega_mle):     5.46370e-08
--------------------------------------------------------------------
Results for equation Close
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.000199         0.000242            0.820           0.412
L1.Close        -0.013115         0.030928           -0.424           0.672
L1.norm          0.001283         0.003507            0.366           0.715
L2.C

In [19]:
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(results.resid)

for col, val in zip(df.columns, out):
    print(col, ':', round(val, 2))

Close : 2.0
norm : 1.99


In [20]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df_train_transformed, variables = df_train_transformed.columns)

,Close_x,norm_x
Close_y,1.0000,0.382
norm_y,0.0152,1.000


In [21]:
lag_order = results.k_ar
print(lag_order)

2


In [22]:
df_input = df_train_transformed.values[-lag_order:]
df_forecast = results.forecast(y=df_input, steps=n_obs)
df_forecast = (pd.DataFrame(df_forecast, index=df_test.index, columns=df_test.columns + '_pred'))

In [23]:
def invert_transformation(df, pred):
    forecast = df_forecast.copy()
    columns = df.columns
    for col in columns:
        forecast[str(col)+'_pred'] = df[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast
output = invert_transformation(df_train, df_forecast)

In [25]:
combined = pd.concat([output['norm_pred'], df_test['norm'], output['Close_pred'], df_test['Close']], axis=1)
combined

,norm_pred,norm,Close_pred,Close
2020-02-16 21:00:00,0.092134,0.078472,-0.003377,-0.012611
2020-02-16 22:00:00,0.152996,0.081920,-0.003087,0.002056
2020-02-16 23:00:00,0.218035,0.037361,-0.002817,0.004192
2020-02-17 00:00:00,0.284959,0.035954,-0.002541,-0.005162
2020-02-17 01:00:00,0.352766,0.061903,-0.002263,0.003647
2020-02-17 02:00:00,0.420992,0.077875,-0.001984,-0.002316
2020-02-17 03:00:00,0.489417,0.108123,-0.001704,0.012125
2020-02-17 04:00:00,0.557937,0.127203,-0.001424,0.004009
2020-02-17 05:00:00,0.626504,0.134718,-0.001143,-0.005570
2020-02-17 06:00:00,0.695093,0.120493,-0.000863,-0.000949


In [27]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


rmse = mean_squared_error(combined['Close_pred'], combined['Close'], squared=False)
mae = mean_absolute_error(combined['Close_pred'], combined['Close'])

print('Forecast accuracy of Close')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Close
RMSE:  0.01
MAE:  0.0
